In [1]:
import spacy
from spacy.matcher import DependencyMatcher

nlp = spacy.load("en_core_web_trf")

pattern1 = [
  {
    "RIGHT_ID": "object",
    "RIGHT_ATTRS": {"POS": "NOUN"}
  },
  # founded -> subject
  {
    "LEFT_ID": "object",
    "REL_OP": ">",
    "RIGHT_ID": "modifier",
    # "RIGHT_ATTRS": {"DEP": {"IN": ["amod", "nummod", "compound", "conj"]}}
    "RIGHT_ATTRS": {"POS": {"IN": ["ADJ"]}}
  }
]
pattern2 = [ 
  {
  "RIGHT_ID": "object",
  "RIGHT_ATTRS": {"POS": "NOUN"}
  },
  # founded -> subject
  {
    "LEFT_ID": "object",
    "REL_OP": ">",
    "RIGHT_ID": "modifier",
    "RIGHT_ATTRS": {"DEP": {"IN": ["amod", "nummod", "compound", "conj"]}}
  },
  {
    "LEFT_ID": "modifier",
    "REL_OP": ">",
    "RIGHT_ID": "modifier2",
    "RIGHT_ATTRS": {"DEP": {"IN": ["conj"]}}
  }
]
pattern3 = [
  {
  "RIGHT_ID": "object",
  "RIGHT_ATTRS": {"POS": "NOUN"}
  },
  # object <-(nsubj)- aux
  {
    "LEFT_ID": "object",
    "REL_OP": "<",
    "RIGHT_ID": "modifier",
    "RIGHT_ATTRS": {"DEP": "prep"}
  },
  {
    "LEFT_ID": "modifier",
    "REL_OP": "<",
    "RIGHT_ID": "modifier2",
    "RIGHT_ATTRS": {"DEP": "acomp"}
  },
  {
    "LEFT_ID": "modifier2",
    "REL_OP": "<",
    "RIGHT_ID": "modifier3",
    "RIGHT_ATTRS": {"DEP": "ccomp"}
  },
  {
    "LEFT_ID": "modifier3",
    "REL_OP": ">",
    "RIGHT_ID": "modifier4",
    "RIGHT_ATTRS": {"DEP": "nsubj"}
  }
]


text = "Something is creating tunnels on my tomato plants. When I cut them open, I see yellowish worms or larvae with red or purple areas."
text2 = "I have an artichoke plant and it has all these little black specks on it! There was a ladybug on topside of leaf. What are they?"
text3 = "We have an almond tree that is producing almonds. Come to find out the almonds are full of worms. Can you help with a what type of worm it is and possibly a plan of action?"
text4 = "The plants are wilting"
doc = nlp(text4)

matcher = DependencyMatcher(nlp.vocab, validate = True)
matcher.add("FOUNDED1", [pattern1])
matcher.add("FOUNDED2", [pattern2])
matcher.add("FOUNDED3", [pattern3])
matches = matcher(doc)

# Each token_id corresponds to one pattern dict
# from spacy import displacy
# displacy.render(doc, style='dep')

for (match_id, token_ids) in matches:
  for i in range(len(token_ids)):
    print(f'{matcher.get(match_id)[1][0][i]["RIGHT_ID"] + ":":<10} {doc[token_ids[i]].text:<10}', end = '\t')
  print()

In [2]:
print(f'{"text":<10} {"lemma_":<10} {"pos_":<10} {"tag_":<10} {"dep_":<10} {"head.text":<10} {"head.pos_":<10} {"children":<10}')
for token in doc:
    print(f'{token.text:<10} {token.lemma_:<10} {token.pos_:<10} {token.tag_:<10} {token.dep_:<10} {token.head.text:<10} {token.head.pos_:<10} {[child for child in token.children]}')

text       lemma_     pos_       tag_       dep_       head.text  head.pos_  children  
The        the        DET        DT         det        plants     NOUN       []
plants     plant      NOUN       NNS        nsubj      wilting    VERB       [The]
are        be         AUX        VBP        aux        wilting    VERB       []
wilting    wilt       VERB       VBG        ROOT       wilting    VERB       [plants, are]


In [3]:
from spacy import displacy

text = 'I have tried several "organic" solutions to treat a nagging aphid population on my Zuccinni squash and my basil. Is there a knock out punch for these pest! Preferably organic.'
doc = nlp(text)

displacy.render(doc, style='dep')

In [9]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
    
            [child for child in token.children])

Something nsubj creating VERB []
is aux creating VERB []
creating ROOT creating VERB [Something, is, tunnels, on, .]
tunnels dobj creating VERB []
on prep creating VERB [plants]
my poss plants NOUN []
tomato compound plants NOUN []
plants pobj on ADP [my, tomato]
. punct creating VERB []
When advmod cut VERB []
I nsubj cut VERB []
cut advcl see VERB [When, I, them, open]
them dobj cut VERB []
open oprd cut VERB []
, punct see VERB []
I nsubj see VERB []
see ROOT see VERB [cut, ,, I, worms, .]
yellowish amod worms NOUN []
worms dobj see VERB [yellowish, or, larvae, with]
or cc worms NOUN []
larvae conj worms NOUN []
with prep worms NOUN [areas]
red amod areas NOUN [or, purple]
or cc red ADJ []
purple conj red ADJ []
areas pobj with ADP [red]
. punct see VERB []
